In [20]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torchvision import transforms
import torch.nn.functional as F
from torcheval.metrics.functional import multiclass_f1_score
from datasets import load_dataset

In [14]:
cifar = load_dataset("cifar10",)
i = cifar["train"]["img"][0]


In [15]:
labels = cifar["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [16]:
class NaturalSceneClassification(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(3, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(32,64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(128 ,128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256,256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            nn.Linear(4096,1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    
    def forward(self, xb):
        return self.network(xb)

In [17]:
transform = transforms.Compose([
    # you can add other transformations in this list
    transforms.ToTensor()
])

class Dataset(Dataset):
    
    # Constructor 
    def __init__(self, X_data, Y_data, transform=transform):
        self.len = len(X_data)
        self.x = X_data
        self.y = Y_data
        self.transform = transform
             
    # Getter
    def __getitem__(self, index):
        x = self.x[index] 
        y = self.y[index]
        if self.transform:
            x = self.transform(x)     
        return x, y
    
    # Get Length
    def __len__(self):
        return self.len


In [18]:
dataset_train = Dataset( X_data = cifar["train"]["img"], Y_data = cifar["train"]["label"])
train_loader = DataLoader(dataset=dataset_train, batch_size=128, shuffle=True)
dataset_test = Dataset( X_data = cifar["test"]["img"], Y_data = cifar["test"]["label"])
test_loader = DataLoader(dataset=dataset_test, batch_size=128, shuffle=True)

In [21]:
torch.cuda.is_available()

True

In [22]:
model = NaturalSceneClassification()
model.to("cuda:0")
#print(model)

import torch
#model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#criterion = nn.MSELoss(reduction="mean")
criterion = nn.BCEWithLogitsLoss()

NaturalSceneClassification(
  (network): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Flatten(start_dim=1, end_dim=-1)
    (16): Linear(in_features=4096, out_features=1024, bias=True)
    (17): ReLU()
    (18): Lin

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


loss_new = nn.CrossEntropyLoss()

model.train()
for epoch in range(4):
    losses = []
    epoch_acc = 0
    epoch_loss = 0
    train_losses = []
    train_acc = []
    train_f1 = []
    for x,y in train_loader:

        optimizer.zero_grad()
        
        #x = x.float()
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        #y = y.float()
        #y = torch.unsqueeze(y, dim=0)
        #y = torch.permute(y,(1,0))
        #print(y.dtype)
        #x = torch.unsqueeze(x, dim=0)
        #print(x.size())
        #x = torch.permute(x,(1, 0, 2))
        #print(x.size())
        #print(y)
        #print("*****************")
        #print(x.size())
        output = model(x)
        #output = output.long()
        #print(output.dtype)
        #print(output.size())
        loss = loss_new(output, y)
        acc = accuracy(output, y)
        f1 = multiclass_f1_score(output, y, num_classes=10)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
        train_acc.append(acc.item())
        train_f1.append(f1.item())
        #epoch_loss += loss.item()
        #epoch_acc += acc.item()
    
    print("epoch loss", sum(train_losses)/len(train_losses), "acc", sum(train_acc)/len(train_acc), sum(train_f1)/len(train_f1))


In [ ]:
model.eval()
with torch.no_grad():
    y_hat = []
    epoch_loss = 0
    for x,y in test_loader:
        
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        #x = torch.unsqueeze(x, dim=0)
        #x = torch.permute(x,(1, 0, 2))
        #y = torch.unsqueeze(y, dim=0)
        #y = torch.permute(y,(1,0))
        #print(x.shape)
        output = model(x)
        #print(output.shape)
        #print(output.shape)
        #output = torch.sigmoid(output)
        #output = torch.round(output)
        
        loss = loss_new(output, y)
        epoch_loss += loss.item()
        #y_hat.append(output)
        #print(y_hat.shape)
        #Acc = metric(output, y)
        #f1 = metric1(output, y)
        #f2 = metric2(output, y)
        #Precision = metric2(output, y)

        #precision = metric3(output, y)
        #print(acc)
        #confmat = torchmetrics.ConfusionMatrix(task="binary", num_classes=2)
        #confmat(output, y)
        #print(confmat)
    print(epoch_loss)